# Installation

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gradio
!pip install -q pyChatGPT

     |████████████████████████████████| 5.8 MB 15.3 MB/s 
     |████████████████████████████████| 7.6 MB 65.1 MB/s 
     |████████████████████████████████| 182 kB 76.9 MB/s 
     |████████████████████████████████| 11.6 MB 12.6 MB/s 
     |████████████████████████████████| 55 kB 4.7 MB/s 
     |████████████████████████████████| 2.3 MB 62.8 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 278 kB 76.2 MB/s 
     |████████████████████████████████| 106 kB 71.6 MB/s 
     |████████████████████████████████| 213 kB 55.8 MB/s 
     |████████████████████████████████| 56 kB 5.6 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 64 kB 3.5 MB/s 
     |████████████████████████████████| 80 kB 12.0 MB/s 
     |████████████████████████████████| 68 kB 8.4 MB/s 
     |████████████████████████████████| 68 kB 8.3 MB/s 
     |████████████████████████

# Imports

In [ ]:
import whisper
import gradio as gr 
import time
from pyChatGPT import ChatGPT
import warnings

# Defining Variables

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
secret_token = "" # Enter your session token here!

In [ ]:
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:06<00:00, 23.7MiB/s]


In [ ]:
model.device

device(type='cuda', index=0)

# Transcribe Function

In [ ]:
def transcribe(audio):

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    result_text = result.text

    # Pass the generated text to Audio
    chatgpt_api = ChatGPT(secret_token)
    resp = chatgpt_api.send_message(result_text)
    out_result = resp['message']

    return [result_text, out_result]

# Gradio Interface

In [ ]:
output_1 = gr.Textbox(label="Speech to Text")
output_2 = gr.Textbox(label="ChatGPT Output")


gr.Interface(
    title = 'OpenAI Whisper and ChatGPT ASR Gradio Web UI', 
    fn=transcribe, 
    inputs=[
        gr.inputs.Audio(source="microphone", type="filepath")
    ],

    outputs=[
        output_1,  output_2
    ],
    live=True).launch()

Hint: Set streaming=True for Audio component to use live streaming.
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>